In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.8 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
     |████████████████████████████████| 596 kB 65.1 MB/s 
     |████████████████████████████████| 6.6 MB 54.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install alibi_detect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 390 kB 4.7 MB/s 
     |████████████████████████████████| 3.3 MB 43.4 MB/s 
     |████████████████████████████████| 34.5 MB 1.3 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.39.0
    Uninstalling llvmlite-0.39.0:
      Successfully uninstalled llvmlite-0.39.0
  Attempting uninstall: numba
    Found existing installation: numba 0.56.0
    Uninstalling numba-0.56.0:
      Successfully uninstalled numba-0.56.0


In [ ]:
import numpy as np
import os
import tensorflow as tf
from transformers import AutoTokenizer
from alibi_detect.cd import KSDrift, MMDDrift
from alibi_detect.saving import save_detector, load_detector

In [ ]:
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
import pandas as pd
import numpy as np
import re
import collections
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
trained_df = pd.read_csv("/content/spam_assasin.csv", encoding='latin-1')
prod_df = pd.read_csv("/content/spam.csv", encoding='latin-1')
prod_df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1, inplace=True)
prod_df.columns = ["SpamHam","sms"]
trained_df.drop(["Unnamed: 0"], axis=1, inplace=True)
trained_df.columns = ["sms","SpamHam"]

In [ ]:
trained_df


,sms,SpamHam
0,"LazyTraders,\n\n\n\nEven when the Market takes...",spam
1,This is a multi-part message in MIME format.\n...,spam
2,URL: http://jeremy.zawodny.com/blog/archives/0...,ham
3,--==_Exmh_-1590407866P\n\nContent-Type: text/p...,ham
4,...see subject\n\n\n\n-- \n\nregards from Germ...,ham
...,...,...
6844,>>From the Hoax or Hack Dept:\n\nDigital Needl...,ham
6845,"URL: http://www.newsisfree.com/click/-2,841882...",ham
6846,\n\nHello I am your hot lil horny toy.\n\n ...,spam
6847,"URL: http://www.newsisfree.com/click/-2,865571...",ham


In [ ]:
prod_df

,SpamHam,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 4.9 MB/s 
     |████████████████████████████████| 106 kB 74.1 MB/s 


In [ ]:
import contractions

In [ ]:
trained_df["sms"]

0       LazyTraders,\n\n\n\nEven when the Market takes...
1       This is a multi-part message in MIME format.\n...
2       URL: http://jeremy.zawodny.com/blog/archives/0...
3       --==_Exmh_-1590407866P\n\nContent-Type: text/p...
4       ...see subject\n\n\n\n-- \n\nregards from Germ...
                              ...                        
6844    >>From the Hoax or Hack Dept:\n\nDigital Needl...
6845    URL: http://www.newsisfree.com/click/-2,841882...
6846    \n\nHello I am your hot lil horny toy.\n\n    ...
6847    URL: http://www.newsisfree.com/click/-2,865571...
6848    \n\n//this function should print all numbers u...
Name: sms, Length: 6849, dtype: object

In [ ]:
lem = WordNetLemmatizer()
def preprocessing(df):
      df=str(df)
      sms = contractions.fix(df) # converting shortened words to original (Eg:"I'm" to "I am")
      sms = sms.lower() # lower casing the sms
      sms = re.sub(r'https?://S+|www.S+', "", sms).strip() #removing url
      sms = re.sub("[^a-z ]", "", sms) # removing symbols and numbers
      sms = sms.split() #splitting
      # lemmatization and stopword removal
      sms = [lem.lemmatize(word) for word in sms if not word in set(stopwords.words("english"))]
      sms = " ".join(sms)
      return sms
X_t = trained_df["sms"].apply(preprocessing)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb_enc = LabelEncoder()
y_t = lb_enc.fit_transform(trained_df["SpamHam"])

In [ ]:
X_prod = prod_df["sms"].apply(preprocessing)
y_prod = lb_enc.fit_transform(prod_df["SpamHam"])

In [ ]:
def random_sample(X: np.ndarray, y: np.ndarray, proba_zero: float, n: int):
    if len(y.shape) == 1:
        idx_0 = np.where(y == 0)[0]
        idx_1 = np.where(y == 1)[0]
    else:
        idx_0 = np.where(y[:, 0] == 1)[0]
        idx_1 = np.where(y[:, 1] == 1)[0]
    
    n_0, n_1 = int(n * proba_zero), int(n * (1 - proba_zero))
    
    idx_0_out = np.random.choice(idx_0, n_0, replace=False)
    idx_1_out = np.random.choice(idx_1, n_1, replace=False)
    X_out = np.concatenate([X[idx_0_out], X[idx_1_out]])
    y_out = np.concatenate([y[idx_0_out], y[idx_1_out]])
    return X_out.tolist(), y_out.tolist()

In [ ]:
X_t

0       lazytraderseven market take odd point dive sti...
1       multipart message mime formatnextpartbddbcaecd...
2       url httpjeremyzawodnycomblogarchiveshtmldate t...
3       exmhpcontenttype textplain charsetusascii scot...
4       see subject regard germany matthiasrpmlist mai...
                              ...                        
6844    hoax hack deptdigital needle virtual gramophon...
6845    url httpwwwnewsisfreecomclickdate tart longrun...
6846    hello hot lil horny toy one dream open minded ...
6847    url httpwwwnewsisfreecomclickdate tbusiness lo...
6848    function print number void printnums int fori ...
Name: sms, Length: 6849, dtype: object

In [ ]:
n_sample = 1000
max_len=100

X_ref = random_sample(X_t, y_t, proba_zero=0.5, n=n_sample)[0]
X_h0 = random_sample(X_t, y_t, proba_zero=.5, n=n_sample)[0]
X_prod_test = random_sample(X_prod, y_prod, proba_zero=.5, n=n_sample)[0]

In [ ]:
from alibi_detect.models.tensorflow import TransformerEmbedding

emb_type = 'hidden_state'
n_layers = 8
layers = [-_ for _ in range(1, n_layers + 1)]

embedding = TransformerEmbedding(model_name, emb_type, layers)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
tokens = tokenizer(list(X_t[:5]), pad_to_max_length=True,
                   max_length=max_len, return_tensors='tf')
x_emb = embedding(tokens)
tf.random.set_seed(0)
from alibi_detect.cd.tensorflow import UAE

enc_dim = 32
shape = (x_emb.shape[1],)

uae = UAE(input_layer=embedding, shape=shape, enc_dim=enc_dim)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
from functools import partial
from alibi_detect.cd.tensorflow import preprocess_drift

# define preprocessing function
preprocess_fn = partial(preprocess_drift, model=uae, tokenizer=tokenizer,
                        max_len=max_len, batch_size=32)

# initialize detector
cd = KSDrift(X_ref, p_val=.05, preprocess_fn=preprocess_fn, input_shape=(max_len,))

In [ ]:
preds_h0 = cd.predict(X_h0)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_h0['data']['is_drift']]))
print('p-value: {}'.format(preds_h0['data']['p_val']))

Drift? No!
p-value: [0.5976715  0.5242368  0.99219006 0.9859872  0.42175108 0.9645013
 0.9645013  0.8789875  0.9055831  0.6729558  0.42175108 0.08365297
 0.7830867  0.05243463 0.97690445 0.99219006 0.8494083  0.63523775
 0.4546595  0.23336123 0.5605799  0.23336123 0.74731666 0.36023447
 0.33175027 0.63523775 0.8789875  0.17527203 0.81728154 0.30482334
 0.0464214  0.5976715 ]


In [ ]:

preds_prod_data = cd.predict(X_prod_test)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_prod_data['data']['is_drift']]))
print('p-value: {}'.format(preds_prod_data['data']['p_val']))

Drift? Yes!
p-value: [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 3.6958096e-34 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 4.3795202e-19 0.0000000e+00 1.4151504e-20 1.2284574e-19 0.0000000e+00
 4.7117847e-04 1.0112726e-18 0.0000000e+00 0.0000000e+00 1.0296152e-07
 2.5097010e-23 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 2.0338660e-10 0.0000000e+00]
